In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from poisson_binomial import poisson_binomial_pmf as direct_convolution_local, poisson_binomial_pmf_direct
from scipy import stats,special
%config Completer.use_jedi = False

In [9]:
X = np.random.randn(35,9000000)

In [10]:
%time np.exp(X)

CPU times: user 4.63 s, sys: 1.8 s, total: 6.42 s
Wall time: 9.1 s


array([[1.05643266, 2.69978103, 6.08422988, ..., 0.93736925, 0.95129011,
        2.53220766],
       [0.75824069, 1.00701729, 4.55163893, ..., 4.94503963, 0.26170908,
        1.59518203],
       [1.56680859, 0.09618056, 4.43471963, ..., 0.54629671, 1.06026497,
        2.2896789 ],
       ...,
       [0.90479423, 0.31745199, 2.83416664, ..., 0.85786445, 0.09455074,
        2.77031359],
       [1.23969285, 1.36455626, 1.6176306 , ..., 2.30245093, 0.62105088,
        2.06622363],
       [5.74533324, 1.19778856, 0.1476776 , ..., 2.87227591, 1.74266333,
        0.73807681]])

# Log Prior

In [ ]:
from numba import njit
from math import ceil

@njit
def func_invKgYinvKt(Ng,Nt,lt,invKgY,invKgYinvKt):

    u = np.exp(-1.0/lt)
    u2 = u*u
    oneplusu2 = 1.0 + u2
    oneoveroneminusu2 = 1.0 / ( 1.0 - u2 )
    
    for ig in range(Ng):
        invKgYinvKt[ig,0] = (invKgY[ig,0] - u * invKgY[ig,1])*oneoveroneminusu2
        invKgYinvKt[ig,-1] = (invKgY[ig,-1] - u * invKgY[ig,-2])*oneoveroneminusu2
        for it in range(1,Nt-1):
            invKgYinvKt[ig,it] = ( oneplusu2 * invKgY[ig,it] - u * ( invKgY[ig,it-1] + invKgY[ig,it+1] ) )*oneoveroneminusu2
            
            
@njit
def func_YJt(Ng,Nt,lt,Y,result,log_eps=np.log(1e-16),safety_margin=10):
    
    u = np.exp(-1/lt)
    u2 = u*u
    
    M = safety_margin+ceil(-lt*log_eps)
    
    power_u = np.power(u,np.arange(M))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        
        for l in range(Nt):
            
            res =  - (1.0+u2)*Y[i,l]/(1.0-u2)
            
            if l < M:
                res += u2*Y[i,0]*power_u[l]/(1.0-u2)
                
            if Nt-l-1 < M:
                res += u2*Y[i,-1]*power_u[Nt-l-1]/(1.0-u2)
            
            #res = (u2*( Y[i,0]*power_u[min(M,l)] + Y[i,-1]*power_u[min(M,Nt-l-1)] ) - (1.0+u2)*Y[i,l])/(1.0-u2)
            
            for j in range(max(0,l-M),min(Nt,l+M)):
                res += Y[i,j]*power_u[abs(l-j)]
                
            result[i,l] = res/lt2
    

def log_prior_x(x,mu,lt,lg,sigma2,G,Ng,Nt):
    
    # X is a (Ng,Nt) matrix
    # mu is a (Ng,) vector
    # sigma2, lt and lg are scalars
    # G2 is a (Ng,Ng) matrix
    
    Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
    
    G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])
    Kg = np.exp(-G2/lg/lg/2.0)
    dKg_dlg = G2*Kg/lg/lg/lg
    Jg = np.linalg.lstsq(np.dot(Kg.T,Kg),np.dot(Kg.T,dKg_dlg),rcond=None)[0]
    invKgY = np.linalg.lstsq(np.dot(Kg.T,Kg),np.dot(Kg.T,Y),rcond=None)[0]
    
    
    invKgYinvKt = np.zeros((Ng,Nt))
    func_invKgYinvKt(Ng,Nt,lt,invKgY,invKgYinvKt)
    
    u = np.exp(-1.0/lt)
    u2 = u*u
    logdetKg = np.linalg.slogdet(Kg)[1]
    logdetinvKt = -(Nt-1.0)*np.log( 1.0 - u2 )
    TrJg =  np.trace(Jg)
    TrJt = -2.0*(Nt-1.0)*u2/(1.0-u2)/lt/lt
    
    Y_invKgYinvKt = (Y*invKgYinvKt).sum()
    
    JgTY = np.dot(Jg.T,Y)
    JgTY_invKgYinvKt = (JgTY*invKgYinvKt).sum()
    
    YJt = np.zeros((Ng,Nt))
    func_YJt(Ng,Nt,lt,Y,YJt,log_eps=np.log(1e-16))
    YJt_invKgYinvKt = (YJt*invKgYinvKt).sum()
    
    lnP = -Ng*Nt*np.log(2.0*np.pi*sigma2)/2.0 + Ng*logdetinvKt/2.0 - Nt*logdetKg/2.0 - Y_invKgYinvKt/2.0/sigma2
    
    dlnP_dX = -invKgYinvKt/sigma2
    dlnP_dx = dlnP_dX.ravel()
    dlnP_dmu = -dlnP_dX.sum(axis=1)
    dlnP_dsigma2 = -Ng*Nt/2.0/sigma2 + Y_invKgYinvKt/2.0/sigma2/sigma2
    dlnP_dlt = -Ng*TrJt/2.0 + YJt_invKgYinvKt/2.0/sigma2
    dlnP_dlg = -Nt*TrJg/2.0 + JgTY_invKgYinvKt/2.0/sigma2
    #print(invKgY)
    #print(Kg,Y)
    
    return lnP, dlnP_dx, dlnP_dmu, dlnP_dlt, dlnP_dlg, dlnP_dsigma2

def log_prior_mu(mu,m,tau2):
    
    J = tau2*np.exp(-G_squared/2.0/m**2)
    J_inv = np.linalg.pinv(J)
    J_inv_mu = np.dot(J_inv,mu)
    
    lnQ = +0.5*np.linalg.slogdet(J_inv)[1]-0.5*np.dot(mu.T,J_inv_mu) - (M/2)*np.log(2.0*np.pi)
    
    dJdm = J*G2/m**3
    dJdtau2 = J/tau2

    dlnQdm = -0.5*np.trace(np.dot(J_inv,dJdm))+0.5*np.dot(J_inv_mu.T,np.dot(dJdm,J_inv_mu))
    dlnQdtau2 = -0.5*np.trace(np.dot(J_inv,dJdtau2))+0.5*np.dot(J_inv_mu.T,np.dot(dJdtau2,J_inv_mu))
    dlnQdmu = -J_inv_mu
    
    return lnQ, dlnQdmu, dlnQdm, dlnQdtau2

def log_prior_lengthscale(l):
    
    # InvGamma(1,2)
    
    lnA = np.log(2.0)-2.0*np.log(l)-2.0/l
    
    dlnAdl = 2.0*(1.0-l)/l/l
    
    return lnA, dlnAdl

def log_prior_variance(sigma2):
    
    # Gamma(1,1)
    
    lnS = - sigma2
    
    dlnSdsigma2 = - 1.0
    
    return lnS, dlnSsigma2

def log_likelihood(p):
    
    return 0.0, np.zeros(p.shape)

def log_posterior(args,Nh,Ng,Nt,G):
    
    # Unpack parameters
    ln_lt, ln_lg, ln_sigma2, ln_m, ln_tau2 = args[:Nh]
    mu = args[Nh:Nh+Ng]
    x = args[Nh+Ng:]
    
    # Transform parameters
    lt, lg, sigma2, m, tau2 = np.exp(ln_lg), np.exp(ln_lt), np.exp(ln_sigma2), np.exp(ln_m), np.exp(ln_tau2)
    p = 1.0/(1.0+np.exp(-x))
    
    ##### Priors
    
    ### Initialise values
    
    lnF = 0.0
    dlnF_dlt, dlnF_dlg, dlnF_dm = 0.0, 0.0
    dlnF_dsigma2, dlnF_dtau2 = 0.0, 0.0
    dlnF_dmu = np.zeros(Ng)
    dlnF_dx = np.zeros(Ng*Nt)
    dlnF_dp = np.zeros(Ng*Nt)
    
    
    ### Prior on x
    
    # Calculate quantities
    lnP, dlnP_dx, dlnP_dmu, dlnP_dlt, dlnP_dlg,dlnP_dsigma2 = log_prior_x(x,mu,lt,lg,sigma2,G,Ng,Nt)

    # Increment log posterior
    lnF += lnP

    # Increment gradients
    dlnF_dx += dlnP_dx
    dlnF_dmu += dlnP_dmu
    dlnF_dlt += dlnP_dlt
    dlnF_dlg += dlnP_dlg
    dlnF_dsigma2 += dlnP_dsigma2
    
    ### Prior on mu
    
    # Calculate quantities
    lnQ, dlnQ_dmu, dlnQ_dm, dlnQ_dtau2 = log_prior_mu(mu,m,tau2)
    
    # Increment log posterior
    lnF += lnQ
    
    # Increment gradients
    dlnF_dmu += dlnQ_dmu
    dlnF_dm += dlnQ_dm
    dlnF_dtau2 += dlnQ_dtau2
    
    ### Prior on lt, lg and m
    
    # Calculate quantities
    lnA, dlnA_dlt = log_prior_lengthscale(lt)
    lnB, dlnB_dlg = log_prior_lengthscale(lg)
    lnC, dlnC_dm  = log_prior_lengthscale(m)
    
    # Increment log posterior
    lnF += lnA
    lnF += lnB
    lnF += lnC
    
    # Increment gradients
    dlnF_dlt += dlnA_dlt
    dlnF_dlg += dlnB_dlg
    dlnF_dm  += dlnC_dm
    
    ### Prior on sigma2 and tau2
    
    # Calculate quantities
    lnS, dlnS_dsigma2 = log_prior_variance(sigma2)
    lnT, dlnT_dtau2 = log_prior_variance(tau2)
    
    # Increment log posterior
    lnF += lnS
    lnF += lnT
    
    # Increment gradients
    dlnF_dsigma2 += dlnS_dsigma2
    dlnF_dtau2 += dlnT_dtau2
    
    ##### Likelihood
    
    # Calculate quantities
    lnL, dlnL_dp = log_likelihood(p)
        
    # Increment log posterior
    lnF += lnL
    
    # Increment gradients
    dlnF_dp += dlnL_dp
    
    ##### Construct gradient with respect to input parameters
    
    # Initialise
    dlnF_dargs = np.zeros(Nh+Ng+Ng*Nt)
    
    # Correct for log-parameterisation
    dlnF_dln_lt = lt*dlnF_dlt
    dlnF_dln_lg = lg*dlnF_dlg
    dlnF_dln_m = m*dlnF_dm
    dlnF_dln_sigma2 = sigma2*dlnF_dsigma2
    dlnF_dln_tau2 = tau2*dlnF_dtau2
    
    # Correct for logit-parameterisation
    dlnF_dx += (1.0/p+1.0/(1.0-p))*dlnF_dp
    
    # Fill in values
    dlnF_dargs[0] = dlnF_dln_lt
    dlnF_dargs[1] = dlnF_dln_lg
    dlnF_dargs[2] = dlnF_dln_sigma2
    dlnF_dargs[3] = dlnF_dln_m
    dlnF_dargs[4] = dlnF_dln_tau2
    dlnF_dargs[Nh:Nh+M] = dlnF_dmu
    dlnF_dargs[Nh+M:] = dlnF_dx
    
    return lnF, dlnFdargs

# Log likelihood

In [50]:
bins = np.array([0.0, 5.0, 16.0, 17.0, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 25.0])
np.digitize(np.nan,bins)-1

35

In [48]:
-np.log(1e-16)*2160

79577.34081387422

In [13]:
p = np.array([0.3,0.6,0.2,0.1,0.9,0.5,0.3,0.2,0.8])
n = p.size
pmf = np.zeros(n+1)
direct_convolution_local(p,n,pmf)
print(pmf,pmf.sum())
print(poisson_binomial_pmf_direct(p))

[1.1289600e-03 1.9156480e-02 1.0565200e-01 2.5434792e-01 3.1159864e-01
 2.0946544e-01 7.9777920e-02 1.6952400e-02 1.8424800e-03 7.7760000e-05] 1.0
[1.1289600e-03 1.9156480e-02 1.0565200e-01 2.5434792e-01 3.1159864e-01
 2.0946544e-01 7.9777920e-02 1.6952400e-02 1.8424800e-03 7.7760000e-05]


In [17]:
from numba import njit
            
@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    direct_convolution_local(probs,probslen,pmf)
    
    likelihood[:] = pmf[k]
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    

k = 0
Ns = 500
subpmf = np.zeros(Ns)
gradient = np.zeros(Ns)
likelihood = np.zeros(1)
poisson_binomial_likelihood(k,p,n,pmf,subpmf,likelihood,gradient)

dp = 1e-4
print(likelihood[0])
print(poisson_binomial_pmf_direct(p)[k])
print(poisson_binomial_pmf_direct(p+dp*np.array([0,0,0,1,0,0,0,0,0]))[k])
print(likelihood[0]+dp*gradient[3])

0.0011289599999999994
0.0011289599999999994
0.0011288345599999997
0.0011288345599999993


In [23]:
from numba import njit
            
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,log_likelihood,gradient):
    
    # Compute the pmf
    direct_convolution_local(probs,probslen,pmf)
    
    likelihood = pmf[k]
    log_likelihood[:] = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = (gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k])/likelihood
        
    

k = 0
Ns = 500
subpmf = np.zeros(Ns)
gradient = np.zeros(Ns)
log_likelihood = np.zeros(1)
poisson_binomial_log_likelihood(k,p,n,pmf,subpmf,log_likelihood,gradient)

dp = 1e-7
print(log_likelihood[0])
print(np.log(poisson_binomial_pmf_direct(p)[k]))
print(log_likelihood[0]+dp*gradient[3])
print(np.log(poisson_binomial_pmf_direct(p+dp*np.array([0,0,0,1,0,0,0,0,0]))[k]))

-6.786458424025957
-6.786458424025957
-6.7864585351370685
-6.786458535137075


In [41]:
from numba import njit
import numpy as np
            
@njit
def poisson_binomial_log_likelihood_truncated(k,c,n,probs,log_likelihood,gradient,pmf,subpmf):
    
    # Assumes that c <= k <= n
    # k is the number of measurements reported by Gaia
    # c is the minimum number of measurements for a star to be reported by Gaia
    # n is the predicted number of visits by Gaia of that star
    # p is the list of measurement probabilities at each visit
    # log_likelihood is where you want the log likelihood to be stored (we can have this be an output of the function instead, if you prefer)
    # gradient is where you want the gradient to be stored (same as above)
    # pmf and subpmf are used for intermediate storage to avoid excessive memory allocation. Set these to be 1000 long and use them for every star.
    
    # Compute the pmf and log_likelihood
    direct_convolution_local(probs,n,pmf)
    
    likelihood = pmf[k]
    correction = 1.0 - np.sum(pmf[0:c])
    log_likelihood[:] = np.log(likelihood) - np.log(correction)
    
    # Branching to set terms in the loop
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == n:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    # Loop over the p's calculating the gradient
    for i in range(n):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,n):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[n-1] = pmf[n]/p
        
        gradient[i] = (gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k])/likelihood - subpmf[c-1]/correction
        
p = np.array([0.3,0.6,0.2,0.1,0.9,0.5,0.3,0.2,0.8])
n = p.size
k = 6
c = 5
Ns = 500
subpmf = np.zeros(Ns)
gradient = np.zeros(Ns)
log_likelihood = np.zeros(1)
poisson_binomial_log_likelihood_truncated(k,c,n,p,log_likelihood,gradient,pmf,subpmf)

dp = 1e-8
print('Predicted log-likelihood at p   ',log_likelihood[0])
print('Predicted log-likelihood at p+dp',log_likelihood[0]+dp*gradient[3])

pmf = poisson_binomial_pmf_direct(p)
likelihood = pmf[k]
correction = 1.0 - np.sum(pmf[0:c])
log_likelihood = np.log(likelihood) - np.log(correction)
print('True log-likelihood at p        ',log_likelihood)

pmf = poisson_binomial_pmf_direct(p+dp*np.array([0,0,0,1,0,0,0,0,0]))
likelihood = pmf[k]
correction = 1.0 - np.sum(pmf[0:c])
log_likelihood = np.log(likelihood) - np.log(correction)
print('True log-likelihood at p+dp     ',log_likelihood)


Predicted log-likelihood at p    -1.3512295610051956
Predicted log-likelihood at p+dp -1.3512295548589086
True log-likelihood at p         -1.3512295610051956
True log-likelihood at p+dp      -1.351229554858909


# Prior on mu - log prior and gradient

In [ ]:
bins = np.array([0.0, 5.0, 16.0, 17.0])
G = 0.5*(bins[1:]+bins[:-1])
G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])
M = bins.size-1

mu = np.array([-1.2,3.4,0.3])
m = 1.2
tau2 = 0.3**2.0


def log_prior(mu,m,tau2):
    J = tau2*np.exp(-G2/2.0/m**2)
    J_inv = np.linalg.pinv(J)
    J_inv_mu = np.dot(J_inv,mu)
    
    lnP = +0.5*np.linalg.slogdet(J_inv)[1]-0.5*np.dot(mu.T,J_inv_mu) - (M/2)*np.log(2.0*np.pi)
    
    dJdm = J*G2/m**3
    dJdtau2 = J/tau2

    dlnPdm = -0.5*np.trace(np.dot(J_inv,dJdm))+0.5*np.dot(J_inv_mu.T,np.dot(dJdm,J_inv_mu))
    dlnPdtau2 = -0.5*np.trace(np.dot(J_inv,dJdtau2))+0.5*np.dot(J_inv_mu.T,np.dot(dJdtau2,J_inv_mu))
    dlnPdmu = -J_inv_mu
    
    return lnP, dlnPdmu, dlnPdm, dlnPdtau2

def check(x,y,z):
    print(x,y,(x-y)/z)

lnP, dlnPdmu, dlnPdm, dlnPdtau2 = log_prior(mu,m,tau2)
lnP_truth = stats.multivariate_normal(mean=np.zeros(M),cov=J,allow_singular=True).logpdf(mu)
check(lnP,lnP_truth,lnP_truth)
h = 1e-7

lnP_dmu = log_prior(mu+h*np.array([1,0,0]),m,tau2)[0]
check(lnP+h*dlnPdmu[0],lnP_dmu,lnP_truth)

lnP_dm = log_prior(mu,m+h,tau2)[0]
check(lnP+h*dlnPdm,lnP_dm,lnP_truth)

lnP_dtau2 = log_prior(mu,m,tau2+h)[0]
check(lnP+h*dlnPdtau2,lnP_dtau2,lnP_truth)

# Prior on x

In [183]:
Ng = 10
Nt = 15
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/10.0
Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))

t = np.arange(Nt)
lt = 10.3
tau = np.abs(t[:,np.newaxis]-t[np.newaxis,:])
K = np.exp(-tau/lt)
invK = np.linalg.pinv(K)
dK_dlt = tau*K/lt/lt
result = np.einsum('ij,jk,kl->il',Y,invK,dK_dlt)
u = np.exp(-1/lt)
u2 = u*u

power_u = np.power(u,np.arange(Nt))

approx = np.zeros((Ng,Nt))

for i in range(Ng):
    for l in range(Nt):
        #approx[i,l] =  (Y[i,0]*np.power(u,l+2) + Y[i,-1]*np.power(u,Nt-l+1) - (1.0+u*u)*Y[i,l])/(1.0-u*u)
        approx[i,l] =  (u2*Y[i,0]*power_u[l] + u2*Y[i,-1]*power_u[Nt-l-1] - (1.0+u2)*Y[i,l])/(1.0-u2)
        for j in range(Nt):
            approx[i,l] += Y[i,j]*power_u[np.abs(l-j)]
approx = approx / (lt*lt)
#print(result)
#print(approx)
np.allclose(result,approx)

True

In [342]:
from numba import njit
from math import ceil

@njit
def func_approx(Ng,Nt,lt,Y,result):
    
    u = np.exp(-1/lt)
    u2 = u*u
    power_u = np.power(u,np.arange(Nt))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        for l in range(Nt):
            res =  (u2*Y[i,0]*power_u[l] + u2*Y[i,-1]*power_u[Nt-l-1] - (1.0+u2)*Y[i,l])/(1.0-u2)
            for j in range(Nt):
                res += Y[i,j]*power_u[abs(l-j)]
            result[i,l] = res/lt2
            
@njit
def func_truncate(Ng,Nt,lt,Y,result,log_eps=np.log(1e-16),safety_margin=10):
    
    u = np.exp(-1/lt)
    u2 = u*u
    
    M = safety_margin+ceil(-lt*log_eps)
    
    power_u = np.power(u,np.arange(M))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        
        for l in range(Nt):
            
            res =  - (1.0+u2)*Y[i,l]/(1.0-u2)
            
            if l < M:
                res += u2*Y[i,0]*power_u[l]/(1.0-u2)
                
            if Nt-l-1 < M:
                res += u2*Y[i,-1]*power_u[Nt-l-1]/(1.0-u2)
            
            #res = (u2*( Y[i,0]*power_u[min(M,l)] + Y[i,-1]*power_u[min(M,Nt-l-1)] ) - (1.0+u2)*Y[i,l])/(1.0-u2)
            
            for j in range(max(0,l-M),min(Nt,l+M)):
                res += Y[i,j]*power_u[abs(l-j)]
                
            result[i,l] = res/lt2
            
def func_truth(Ng,Nt,lt,Y,result):
    
    t = np.arange(Nt)
    tau = np.abs(t[:,np.newaxis]-t[np.newaxis,:])
    K = np.exp(-tau/lt)
    invK = np.linalg.pinv(K)
    dK_dlt = tau*K/lt/lt
    result[:] = np.einsum('ij,jk,kl->il',Y,invK,dK_dlt)

In [353]:
Ng = 35
Nt = 30000
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/10.0
Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
lt = 10.2
result_approx = np.zeros((Ng,Nt))
result_truncate = np.zeros((Ng,Nt))
result_truth = np.zeros((Ng,Nt))

In [354]:
if Nt < 250:
    %time func_approx(Ng,Nt,lt,Y,result_approx)
    %time func_truncate(Ng,Nt,lt,Y,result_truncate)
    %time func_truth(Ng,Nt,lt,Y,result_truth)
    #%timeit func_approx(Ng,Nt,lt,Y,result_approx)
    print(np.allclose(result_approx,result_truth),np.allclose(result_truncate,result_truth),np.allclose(result_approx,result_truncate))
elif Nt < 30000+1e-10:
    %time func_approx(Ng,Nt,lt,Y,result_approx)
    %time func_truncate(Ng,Nt,lt,Y,result_truncate)
    print(np.allclose(result_approx,result_truncate))
else:
    %time func_truncate(Ng,Nt,lt,Y,result_truncate)

CPU times: user 1min 2s, sys: 599 ms, total: 1min 2s
Wall time: 1min 2s
CPU times: user 1.16 s, sys: 11.2 ms, total: 1.17 s
Wall time: 1.17 s
True


In [709]:
from numba import njit
from math import ceil

@njit
def func_invKgYinvKt(Ng,Nt,lt,invKgY,invKgYinvKt):

    u = np.exp(-1.0/lt)
    u2 = u*u
    oneplusu2 = 1.0 + u2
    oneoveroneminusu2 = 1.0 / ( 1.0 - u2 )
    
    for ig in range(Ng):
        invKgYinvKt[ig,0] = (invKgY[ig,0] - u * invKgY[ig,1])*oneoveroneminusu2
        invKgYinvKt[ig,-1] = (invKgY[ig,-1] - u * invKgY[ig,-2])*oneoveroneminusu2
        for it in range(1,Nt-1):
            invKgYinvKt[ig,it] = ( oneplusu2 * invKgY[ig,it] - u * ( invKgY[ig,it-1] + invKgY[ig,it+1] ) )*oneoveroneminusu2
            
            
@njit
def func_YJt(Ng,Nt,lt,Y,result,log_eps=np.log(1e-16),safety_margin=10):
    
    u = np.exp(-1/lt)
    u2 = u*u
    
    M = safety_margin+ceil(-lt*log_eps)
    
    power_u = np.power(u,np.arange(M))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        
        for l in range(Nt):
            
            res =  - (1.0+u2)*Y[i,l]/(1.0-u2)
            
            if l < M:
                res += u2*Y[i,0]*power_u[l]/(1.0-u2)
                
            if Nt-l-1 < M:
                res += u2*Y[i,-1]*power_u[Nt-l-1]/(1.0-u2)
            
            #res = (u2*( Y[i,0]*power_u[min(M,l)] + Y[i,-1]*power_u[min(M,Nt-l-1)] ) - (1.0+u2)*Y[i,l])/(1.0-u2)
            
            for j in range(max(0,l-M),min(Nt,l+M)):
                res += Y[i,j]*power_u[abs(l-j)]
                
            result[i,l] = res/lt2
    

def log_prior_x(x,mu,sigma2,lt,lg,G2,Ng,Nt,lambd=1e-14,flag=True):
    
    # X is a (Ng,Nt) matrix
    # mu is a (Ng,) vector
    # sigma2, lt and lg are scalars
    # G2 is a (Ng,Ng) matrix
    
    Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
    
    if flag:
        Kg = np.exp(-G2/lg/lg/2.0)
        invKg = np.linalg.pinv(Kg)
        dKg_dlg = G2*Kg/lg/lg/lg
        Jg = np.dot(invKg,dKg_dlg)
        invKgY = np.dot(invKg,Y)
    elif False:
        Kg = np.exp(-G2/lg/lg/2.0)+lambd*np.diag(np.ones(Ng))
        invKg = np.linalg.pinv(Kg)
        dKg_dlg = G2*Kg/lg/lg/lg
        Jg = np.dot(invKg,dKg_dlg)
        invKgY = np.dot(invKg,Y)
    else:
        Kg = np.exp(-G2/lg/lg/2.0)
        dKg_dlg = G2*Kg/lg/lg/lg
        #Jg = np.linalg.lstsq(Kg,dKg_dlg,rcond=None)[0]
        #invKgY = np.linalg.lstsq(Kg,Y,rcond=None)[0]
        Jg = np.linalg.lstsq(np.dot(Kg.T,Kg)+lambd*np.diag(np.ones(Ng)),np.dot(Kg.T,dKg_dlg),rcond=None)[0]
        invKgY = np.linalg.lstsq(np.dot(Kg.T,Kg)+lambd*np.diag(np.ones(Ng)),np.dot(Kg.T,Y),rcond=None)[0]
        #np.linalg.lstsq(Kg,Y,rcond=None)[0]
    
    
    invKgYinvKt = np.zeros((Ng,Nt))
    func_invKgYinvKt(Ng,Nt,lt,invKgY,invKgYinvKt)
    
    u = np.exp(-1.0/lt)
    u2 = u*u
    logdetKg = np.linalg.slogdet(Kg)[1]
    logdetinvKt = -(Nt-1.0)*np.log( 1.0 - u2 )
    TrJg =  np.trace(Jg)
    TrJt = -2.0*(Nt-1.0)*u2/(1.0-u2)/lt/lt
    
    Y_invKgYinvKt = (Y*invKgYinvKt).sum()
    
    JgTY = np.dot(Jg.T,Y)
    JgTY_invKgYinvKt = (JgTY*invKgYinvKt).sum()
    
    YJt = np.zeros((Ng,Nt))
    func_YJt(Ng,Nt,lt,Y,YJt,log_eps=np.log(1e-16))
    YJt_invKgYinvKt = (YJt*invKgYinvKt).sum()
    
    lnP = -Ng*Nt*np.log(2.0*np.pi*sigma2)/2.0 + Ng*logdetinvKt/2.0 - Nt*logdetKg/2.0 - Y_invKgYinvKt/2.0/sigma2
    
    dlnP_dX = -invKgYinvKt/sigma2
    dlnP_dx = dlnP_dX.ravel()
    dlnP_dmu = -dlnP_dX.sum(axis=1)
    dlnP_dsigma2 = -Ng*Nt/2.0/sigma2 + Y_invKgYinvKt/2.0/sigma2/sigma2
    dlnP_dlt = -Ng*TrJt/2.0 + YJt_invKgYinvKt/2.0/sigma2
    dlnP_dlg = -Nt*TrJg/2.0 + JgTY_invKgYinvKt/2.0/sigma2
    #print(invKgY)
    #print(Kg,Y)
    
    return lnP, dlnP_dx, dlnP_dmu, dlnP_dsigma2, dlnP_dlt, dlnP_dlg


In [680]:
def log_prior_x_testing(x,mu,sigma2,lt,lg,T,G,Ng,Nt):
    T = np.arange(Nt)
    Xg, Xt = np.meshgrid(G, T, sparse=False, indexing='ij')
    Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
    xg, xt, y = Xg.reshape((Ng*Nt,1)), Xt.reshape((Ng*Nt,1)), Y.reshape((Ng*Nt,1))
    K = sigma2 * np.exp(-np.abs(xt-xt.T)/lt) * np.exp(-np.square(xg-xg.T)/lg/lg/2.0)
    invK = np.linalg.pinv(K)
    lnP = -Ng*Nt*np.log(2.0*np.pi)/2.0 - np.linalg.slogdet(K)[1]/2.0 -0.5*np.dot(y.T,np.dot(invK,y))
    return lnP[0,0]

In [703]:
np.random.seed(1)
g = np.array([0.0, 5.0, 16.0, 17.0, 18.0])#, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 25.0])
G = 0.5*(g[1:]+g[:-1])
G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])
Ng = g.size-1
Nt = 2
T = np.arange(Nt)
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/10.0
#Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
sigma2, lt, lg = 2.2,100.2,1.3
h = 1e-8

%time lnP, dlnP_dx, dlnP_dmu, dlnP_dsigma2, dlnP_dlt, dlnP_dlg = log_prior_x(x,mu,sigma2,lt,lg,G2,Ng,Nt)
#print('done')
lnP_test = log_prior_x_testing(x,mu,sigma2,lt,lg,T,G,Ng,Nt)

print('lnP\t\t',lnP,lnP_test)

lnP_test = log_prior_x_testing(x,mu,sigma2,lt+h,lg,T,G,Ng,Nt)
print('dlnP_dlt\t',lnP+h*dlnP_dlt,lnP_test) 

lnP_test = log_prior_x_testing(x,mu,sigma2,lt,lg+h,T,G,Ng,Nt)
print('dlnP_dlg\t',lnP+h*dlnP_dlg,lnP_test) 

lnP_test = log_prior_x_testing(x,mu,sigma2+h,lt,lg,T,G,Ng,Nt)
print('dlnP_dsigma2\t',lnP+h*dlnP_dsigma2,lnP_test)

lnP_test = log_prior_x_testing(x,mu+h,sigma2,lt,lg,T,G,Ng,Nt)
print('dlnP_dmu\t',lnP+(h*dlnP_dmu).sum(),lnP_test)

lnP_test = log_prior_x_testing(x+h,mu,sigma2,lt,lg,T,G,Ng,Nt)
print('dlnP_dx\t\t',lnP+(h*dlnP_dx).sum(),lnP_test)

CPU times: user 545 ms, sys: 13 ms, total: 558 ms
Wall time: 560 ms
lnP		 -178.64112134459376 -178.6411213445928
dlnP_dlt	 -178.64112136186935 -178.64112136186787
dlnP_dlg	 -178.64112126436964 -178.64112126437163
dlnP_dsigma2	 -178.6411205591856 -178.64112055918696
dlnP_dmu	 -178.641121348297 -178.64112134829605
dlnP_dx		 -178.64112134089052 -178.64112134088958


In [721]:
np.random.seed(1)
g = np.array([0.0, 5.0, 16.0, 17.0, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 25.0])
#g = np.array([16.0, 17.0, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5])#, 25.0])

#g = np.arange(36)
#g[0] -= 100
G = 0.5*(g[1:]+g[:-1])
G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])
Ng = g.size-1
Nt = 2000
T = np.arange(Nt)
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/30.0
#Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
sigma2, lt, lg = 2.2,100.2,0.3

%time lnP, dlnP_dx, dlnP_dmu, dlnP_dsigma2, dlnP_dlt, dlnP_dlg = log_prior_x(x,mu,sigma2,lt,lg,G2,Ng,Nt,lambd=0.0,flag=True)
print(lnP)

%time lnP, dlnP_dx, dlnP_dmu, dlnP_dsigma2, dlnP_dlt, dlnP_dlg = log_prior_x(x,mu,sigma2,lt,lg,G2,Ng,Nt,lambd=0.0,flag=False)
print(lnP)

CPU times: user 226 ms, sys: 2.37 ms, total: 229 ms
Wall time: 230 ms
-3.762887983813031e+18
CPU times: user 223 ms, sys: 1.74 ms, total: 225 ms
Wall time: 226 ms
-49138377449.46614


In [ ]:
-449610475.01928824,-5679575041.718402

In [677]:
import tqdm
lambd = np.logspace(-16,-3,100)
lnP = np.array([log_prior_x(x,mu,sigma2,lt,lg,G2,Ng,Nt,l)[0] for l in tqdm.tqdm(lambd)])

100%|██████████| 100/100 [00:20<00:00,  4.79it/s]


In [628]:
np.linalg.slogdet(np.exp(-G2/lg/lg/2.0)+1e-11*np.diag(np.ones(Ng)))[1]

-398.52740040557495

In [678]:
plt.figure()
plt.plot(lambd,-lnP)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('regularisation')
plt.ylabel('lnP')
plt.savefig('./line.png',dpi=300,bbox_inches='tight')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [711]:
Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
Kg_mod = np.exp(-G2/lg/lg/2.0)+1e-4*np.diag(np.ones(Ng))
invKg_mod = np.linalg.pinv(Kg_mod,hermitian=True)

Kg = np.exp(-G2/lg/lg/2.0)
invKg = np.linalg.pinv(Kg,hermitian=True)
#Kg[Kg<1e-7] = 0.0
#dKg_dlg = G2*Kg/lg/lg/lg
#Jg = np.linalg.lstsq(Kg,dKg_dlg,rcond=None)[0]
#invKgY = np.linalg.lstsq(Kg,Y,rcond=None)[0]
invKg = np.linalg.lstsq(np.dot(Kg.T,Kg)+1e-10*np.diag(np.ones(Ng)),Kg.T,rcond=None)[0]



In [713]:
np.linalg.slogdet(Kg)[1]

-967.0161740445541

In [714]:
np.linalg.slogdet(np.dot(Kg.T,Kg))[1]

-913.5607862529772

In [660]:
r = np.linalg.matrix_rank(Kg)
INVKG = np.dot(vt.T[:,:r],np.dot(np.diag(1.0/s[:r]),u[:,:r].T))

plt.figure()
plt.imshow(np.dot(INVKG,Kg),cmap='RdBu_r',vmin=-0.5,vmax=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [492]:
plt.figure()
plt.imshow(Kg_mod)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [593]:
np.linalg.slogdet(Kg),np.linalg.slogdet(Kg_mod)

((-1.0, -492.09180506832456), (1.0, -182.61204788514593))

In [621]:
plt.figure()
#ax1.imshow(np.diag(np.ones(Ng)),cmap='RdBu_r',vmin=-0.5,vmax=0.5)
plt.imshow(np.dot(invKg,Kg),cmap='RdBu_r',vmin=-0.5,vmax=0.5)
#plt.savefig('./ugly.png',dpi=300,bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [317]:
Lg

array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [2.57220937e-056, 1.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [5.70904011e-171, 5.38018616e-032, 1.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [2.34853971e-309, 5.59159501e-103, 3.70353198e-021, ...,
        4.89248978e-004, 0.00000000e+000, 0.00000000e+000],
       [1.22329619e-312, 7.14515686e-105, 5.21673666e-022, ...,
        3.07543319e-003, 4.88853657e-004, 0.00000000e+000],
       [0.00000000e+000, 6.30966891e-142, 2.66020642e-040, ...,
        3.09297601e-002, 5.21319436e-002, 9.97955107e-001]])

In [327]:
Lg = np.linalg.cholesky(Kg+1e-15*np.diag(np.ones(Ng)))
invLg = np.linalg.pinv(Lg).T#,np.diag(np.ones(Ng)),rcond=None)[0]
invKg = np.dot(invLg,invLg.T)

In [404]:
np.linalg.pinv(np.exp(-G2/lt/lt/2-np.linalg.slogdet(Kg)[1]/Nt))

array([[ 6.23164582e+09, -3.26997790e+09, -3.92248215e+09, ...,
        -2.24738583e+09, -3.32566835e+09, -4.54163622e+09],
       [-3.27027406e+09,  1.71651805e+09,  2.05883595e+09, ...,
         1.17980334e+09,  1.74580904e+09,  2.38408497e+09],
       [-3.92241897e+09,  2.05861636e+09,  2.46924268e+09, ...,
         1.41490314e+09,  2.09372092e+09,  2.85921446e+09],
       ...,
       [-2.24740417e+09,  1.17970613e+09,  1.41493747e+09, ...,
         8.10852263e+08,  1.19984609e+09,  1.63850802e+09],
       [-3.32566996e+09,  1.74565180e+09,  2.09375566e+09, ...,
         1.19983689e+09,  1.77544681e+09,  2.42455295e+09],
       [-4.54140182e+09,  2.38374606e+09,  2.85911295e+09, ...,
         1.63841010e+09,  2.42442665e+09,  3.31080527e+09]])

In [396]:
plt.figure()
plt.imshow(Kg)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [600]:
np.linalg.s

(0.3678794411714422, 0.36787944117144233)

In [606]:
n = 4
l = 1.0/np.sqrt(2)
x = np.arange(n).reshape((n,1))
X2 = np.square(x-x.T)
u = np.exp(-1.0/l/l/2.0)
K = u**X2

In [607]:
print(K)

[[1.00000000e+00 3.67879441e-01 1.83156389e-02 1.23409804e-04]
 [3.67879441e-01 1.00000000e+00 3.67879441e-01 1.83156389e-02]
 [1.83156389e-02 3.67879441e-01 1.00000000e+00 3.67879441e-01]
 [1.23409804e-04 1.83156389e-02 3.67879441e-01 1.00000000e+00]]


In [610]:
1.18102267/0.50123129

2.3562429033510655

In [609]:
invK = np.linalg.pinv(K)
print(invK)

[[ 1.18102267 -0.50123129  0.18439269 -0.05879966]
 [-0.50123129  1.39082    -0.57030799  0.18439269]
 [ 0.18439269 -0.57030799  1.39082    -0.50123129]
 [-0.05879966  0.18439269 -0.50123129  1.18102267]]


In [646]:
invK*(1.0-u**2)**3*(1+u**2)

array([[ 0.86681333, -0.36787944,  0.13533528, -0.04315609],
       [-0.36787944,  1.02079439, -0.41857839,  0.13533528],
       [ 0.13533528, -0.41857839,  1.02079439, -0.36787944],
       [-0.04315609,  0.13533528, -0.36787944,  0.86681333]])

# Investigating priors on hyperparameters

### Lengthscale

In [ ]:
plt.figure()
x = np.linspace(0.1,10,10001)

# InvGamma(5,5)
plt.plot(x,stats.invgamma(a=1.0,scale=2.0).pdf(x))
plt.yscale('log')

### Variance

In [ ]:
plt.figure()
x = np.linspace(0.1,10,10001)

# InvGamma(5,5)
plt.plot(x,stats.gamma(a=1.0,scale=1.0).pdf(x))
plt.yscale('log')

# Global

In [37]:
stats.rv_discrete(values=(np.arange(stars[i]['n']+1),pmf)).rvs()

0

2

In [5]:
poisson_binomial_pmf_direct(np.array([0.4,0.9,0.96]))

array([0.0024, 0.0808, 0.5712, 0.3456])

In [78]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b

In [79]:
c.shape

(3, 3)

In [63]:
#bins = np.array([0.0, 5.0, 16.0, 17.0, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 25.0])
bins = np.array([-2.0,-1.0,0.0, 1.0, 2.0])
G = 0.5*(bins[1:]+bins[:-1])
G_squared = np.square(G[:,np.newaxis]-G[np.newaxis,:])
M = bins.size-1
N = 10
p_groundtruth = special.expit(-G[:,np.newaxis]*np.ones((M,N)))
p_groundtruth[:,:3] = 0.0

stars = {}
times = np.arange(N).astype(np.int)
N_star = 123
for i in range(N_star):
    stars[i] = {}
    stars[i]['n'] = np.random.randint(N)
    stars[i]['g'] = np.random.randint(M)
    stars[i]['t'] = np.sort(np.random.choice(times,stars[i]['n'],replace=False))
    probs = p_groundtruth[stars[i]['g'],stars[i]['t']]
    pmf = poisson_binomial_pmf_direct(probs)
    stars[i]['k'] = np.random.choice(np.arange(0, stars[i]['n']+1), p=pmf)
    
#tmp_args = np.concatenate([np.array([4.0,0.5,4.0,0.5]),np.ones(N_bins),np.random.normal(0,1,N_bins*N_times)])

def log_prior_x(x,mu,l,sigma2):
    
    expml = np.exp(-l)
    expm2l = expml*expml
    
    d = x-mu
    
    A = d[0]**2+d[-1]**2
    B = np.sum(d[1:-1]**2)
    C = d[:-1]*d[1:]
    D = d[0] + d[-1]
    E = np.sum(d[1:-1])
    
    lnP = -N/2*np.log(sigma2) - (N-1)/2*np.log(1-expm2l) - N/2*np.log(2*np.pi) - ( A + (1+expm2l)*B - 2*expml*C )/(2*sigma2*(1-expm2l))
    
    dlnPdl = ((N-1)*sigma2*expm2l*(1-expm2l) + C*expml*(1+expm2l) - expm2l*(A+2*B))/(l*l*(1-expm2l)*(1-expm2l))
    
    dlnPdsigma2 = -N/2/sigma2 + ( A + (1+expm2l)*B - 2*expml*C )/(2*sigma2*sigma2*(1-expm2l))
    
    dlnPdmu =  (D+(1-expml)*E)/(sigma2*(1+xpml))
    
    dlnPdx = np.zeros(N)
    dlnPdx[0] = -(d[0]-expml*d[1])/(sigma2*(1-expm2l))
    for i in range(1,N-1):
        dlnPdx[i] = -((1+expm2l)*d[i]-expml*(d[i-1]+d[i+1]))/(sigma2*(1-expm2l))
    dlnPdx[-1] = -(d[-1]-expml*d[-2])/(sigma2*(1-expm2l))
    
    return lnP, dlnPdx, dlnPdmu, dlnPdl, dlnPdsigma2

def log_prior_mu(mu,m,tau2):
    
    J = tau2*np.exp(-G_squared/2.0/m**2)
    J_inv = np.linalg.pinv(J)
    J_inv_mu = np.dot(J_inv,mu)
    
    lnQ = +0.5*np.linalg.slogdet(J_inv)[1]-0.5*np.dot(mu.T,J_inv_mu) - (M/2)*np.log(2.0*np.pi)
    
    dJdm = J*G2/m**3
    dJdtau2 = J/tau2

    dlnQdm = -0.5*np.trace(np.dot(J_inv,dJdm))+0.5*np.dot(J_inv_mu.T,np.dot(dJdm,J_inv_mu))
    dlnQdtau2 = -0.5*np.trace(np.dot(J_inv,dJdtau2))+0.5*np.dot(J_inv_mu.T,np.dot(dJdtau2,J_inv_mu))
    dlnQdmu = -J_inv_mu
    
    return lnQ, dlnQdmu, dlnQdm, dlnQdtau2

def log_prior_lengthscale(l):
    
    # InvGamma(1,2)
    
    lnA = np.log(2.0)-2.0*np.log(l)-2.0/l
    
    dlnAdl = 2.0*(1.0-l)/l/l
    
    return lnA, dlnAdl

def log_prior_variance(sigma2):
    
    # Gamma(1,1)
    
    lnS = - sigma2
    
    dlnSdsigma2 = - 1.0
    
    return lnS, dlnSsigma2

def log_likelihood(p):
    lnL, dlnLdp = 0.0, np.zeros(M*N)
    
    for star in stars:
        L, dLdp = 0,0

def log_posterior(args):
    
    # Unpack parameters
    lnl, lnsigma2, lnm, lntau2 = args[:4]
    mu = args[4:4+M]
    x = args[4+M:]
    
    # Transform parameters
    l, sigma2, m, tau2 = np.exp(lnl), np.exp(lnsigma2), np.exp(lnm), np.exp(lntau2)
    p = 1.0/(1.0+np.exp(-x))
    
    ##### Priors
    
    ### Initialise values
    
    lnF = 0.0
    dlnFdl ,dlnFdm = 0.0, 0.0
    dlnFdsigma2, dlnFdtau2 = 0.0, 0.0
    dlnFdmu = np.zeros(M)
    dlnFdx = np.zeros(M*N)
    dlnFdp = np.zeros(M*N)
    
    
    ### Prior on x
    
    for g in range(M):
        
        # Calculate quantities
        lnP, dlnPdx, dlnPdmu, dlnPdl, dlnPdsigma2 = log_prior_x(x[g*N:(g+1)*N],mu[g],l,sigma2)
        
        # Increment log posterior
        lnF += lnP
        
        # Increment gradients
        dlnFdx[g*N:(g+1)*N] += dlnPdx
        dlnFdmu[g] += dlnPdmu
        dlnFdl += dlnPdl
        dlnFdsigma2 += dlnPdsigma2
    
    ### Prior on mu
    
    # Calculate quantities
    lnQ, dlnQdmu, dlnQdm, dlnQdtau2 = log_prior_mu(mu,m,tau2)
    
    # Increment log posterior
    lnF += lnQ
    
    # Increment gradients
    dlnFdmu += dlnQdmu
    dlnFdm += dlnQdm
    dlnFdtau2 += dlnQdtau2
    
    
    ### Prior on l and m
    
    # Calculate quantities
    lnA, dlnAdl = log_prior_lengthscale(l)
    lnB, dlnBdm = log_prior_lengthscale(m)
    
    # Increment log posterior
    lnF += lnA
    lnF += lnB
    
    # Increment gradients
    dlnFdl += dlnAdl
    dlnFdm += dlnBdm
    
    ### Prior on sigma2 and tau2
    
    # Calculate quantities
    lnS, dlnSdsigma2 = log_prior_variance(sigma2)
    lnT, dlnTdtau2 = log_prior_variance(tau2)
    
    # Increment log posterior
    lnF += lnS
    lnF += lnT
    
    # Increment gradients
    dlnFdsigma2 += dlnSdsigma2
    dlnFdtau2 += dlnTdtau2
    
    ##### Likelihood
    
    # Calculate quantities
    lnL, dlnLdp = log_likelihood(p)
        
    # Increment log posterior
    lnF += lnL
    
    # Increment gradients
    dlnFdp += dlnLdp
    
    ##### Construct gradient with respect to input parameters
    
    # Initialise
    dlnFdargs = np.zeros(4+M+M*N)
    
    # Correct for log-parameterisation
    dlnFdlnl = l*dlnFdl
    dlnFdlnm = m*dlnFdm
    dlnFdlnsigma2 = sigma2*dlnFdsigma2
    dlnFdlntau2 = tau2*dlnFdtau2
    
    # Correct for logit-parameterisation
    dlnFdx += (1.0/p+1.0/(1.0-p))*dlnFdp
    
    # Fill in values
    dlnFdargs[0] = dlnFdlnl
    dlnFdargs[1] = dlnFdlnsigma2
    dlnFdargs[2] = dlnFdlnm
    dlnFdargs[3] = dlnFdlntau2
    dlnFdargs[4:4+M] = dlnFdmu
    dlnFdargs[4+M:] = dlnFdx
    
    return lnF, dlnFdargs
    

SyntaxError: invalid syntax (<ipython-input-63-2fe86dc56c0d>, line 94)

# Poisson binomial likelihood

In [72]:
p = np.array([0.3,0.6,0.2,0.1,0.9,0.5])
n = p.size
pmf = np.zeros(n+1)
poisson_binomial_pmf(p,n,pmf)
print(pmf,pmf.sum())
print(poisson_binomial_pmf_direct(p))

[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162] 1.0
[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162]


In [66]:
from numba import njit
            
@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood[0] = pmf[k]
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    

k = 5
subpmf = np.zeros(n)
gradient = np.zeros(n)
likelihood = np.zeros(1)
poisson_binomial_likelihood(k,p,n,pmf,subpmf,likelihood,gradient)

dp = 1e-4
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp*np.array([0,0,0,1,0,0]))[k])
print(likelihood[0]+dp*gradient[3])

0.027719999999999998
0.02773152
0.02773152


In [ ]:
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood = pmf[k]
    log_likelihood = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    return log_likelihood

In [ ]:
dp = 1e-7*np.random.uniform(-1,1,6)
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp)[k])
print(likelihood[0]+np.dot(dp,gradient))

In [ ]:
from numba import njit
            
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,log_likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood = pmf[k]
    log_likelihood[0] = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = (gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k])/likelihood
        
    

k = 4
subpmf = np.zeros(n)
gradient = np.zeros(n)
log_likelihood = np.zeros(1)
poisson_binomial_log_likelihood(k,p,n,pmf,subpmf,log_likelihood,gradient)

dp = 1e-7*np.random.uniform(-1,1,6)
print(log_likelihood[0])
print(np.log(poisson_binomial_pmf_easy(p+dp)[k]))
print(log_likelihood[0]+np.dot(dp,gradient))

In [24]:
dKt_dl = tau*K/l/l
print(np.trace(np.dot(K_inv,dKt_dl)))
print(-2*u*u*(N-1)/(1-u*u))
    

-1.2521411419973258
-1.2521411419973254


In [23]:
dKt_dl,u

(array([[0.        , 0.36787944, 0.27067057, 0.14936121, 0.07326256],
        [0.36787944, 0.        , 0.36787944, 0.27067057, 0.14936121],
        [0.27067057, 0.36787944, 0.        , 0.36787944, 0.27067057],
        [0.14936121, 0.27067057, 0.36787944, 0.        , 0.36787944],
        [0.07326256, 0.14936121, 0.27067057, 0.36787944, 0.        ]]),
 0.36787944117144233)

In [19]:
N = 5
x = np.arange(N)
l = 1.0
tau = np.abs(x[:,np.newaxis]-x[np.newaxis,:])
K = np.exp(-tau/l)
K_inv = np.linalg.pinv(K)
K_inv[np.abs(K_inv) < 1e-14] = 0.0
print(K_inv)
print(np.linalg.det(K_inv))
u = np.exp(-1.0/l)
a = 1
b = 1+u*u
c = -u
#print(a,b,c)
J_inv = np.zeros(K.shape)
J_inv[0,0] = J_inv[-1,-1] = a
for i in range(1,N-1):
    J_inv[i,i] = b
for i in range(0,N-1):
    J_inv[i,i+1] = J_inv[i+1,i] = c
J_inv /= 1-u*u
print(J_inv)
print(np.power(1/(1-u*u),N-1))

[[ 1.15651764 -0.42545906  0.          0.          0.        ]
 [-0.42545906  1.31303529 -0.42545906  0.          0.        ]
 [ 0.         -0.42545906  1.31303529 -0.42545906  0.        ]
 [ 0.          0.         -0.42545906  1.31303529 -0.42545906]
 [ 0.          0.          0.         -0.42545906  1.15651764]]
1.7889946812194075
[[ 1.15651764 -0.42545906  0.          0.          0.        ]
 [-0.42545906  1.31303529 -0.42545906  0.          0.        ]
 [ 0.         -0.42545906  1.31303529 -0.42545906  0.        ]
 [ 0.          0.         -0.42545906  1.31303529 -0.42545906]
 [ 0.          0.          0.         -0.42545906  1.15651764]]
1.788994681219407


In [102]:
slinalg.cholesky(G)

LinAlgError: 1-th leading minor of the array is not positive definite

In [99]:
G = np.exp(-tau/l)*tau/l/l
G_inv = np.linalg.pinv(G)
print(G_inv)

[[ 3.92358517  4.59283433  0.79863201 -3.23369341 -5.77084919]
 [ 4.59283433  0.81199902  0.77154032 -1.81199902 -3.23369341]
 [ 0.79863201  0.77154032 -1.69054892  0.77154032  0.79863201]
 [-3.23369341 -1.81199902  0.77154032  0.81199902  4.59283433]
 [-5.77084919 -3.23369341  0.79863201  4.59283433  3.92358517]]


In [29]:
np.linalg.inv(np.linalg.cholesky(G)).T

array([[1.        , 0.36787944, 0.13533528, 0.04978707, 0.01969691],
       [0.        , 1.        , 0.36787944, 0.13533528, 0.05354177],
       [0.        , 0.        , 1.        , 0.36787944, 0.14554161],
       [0.        , 0.        , 0.        , 1.        , 0.39562311],
       [0.        , 0.        , 0.        , 0.        , 1.0754151 ]])

In [48]:
def print_matrix(X):
    X[np.abs(X)<1e-14] = 0.0
    print(X)
print_matrix(np.linalg.cholesky(np.linalg.pinv(K)))
print_matrix(np.linalg.pinv(np.linalg.cholesky(K)))

[[ 1.0754151   0.          0.          0.          0.        ]
 [-0.39562311  1.0754151   0.          0.          0.        ]
 [ 0.         -0.39562311  1.0754151   0.          0.        ]
 [ 0.          0.         -0.39562311  1.0754151   0.        ]
 [ 0.          0.          0.         -0.39562311  1.        ]]
[[ 1.          0.          0.          0.          0.        ]
 [-0.39562311  1.0754151   0.          0.          0.        ]
 [ 0.         -0.39562311  1.0754151   0.          0.        ]
 [ 0.          0.         -0.39562311  1.0754151   0.        ]
 [ 0.          0.          0.         -0.39562311  1.0754151 ]]


In [60]:
from scipy import linalg as slinalg

In [89]:
slinalg.cholesky(inv_X)

array([[ 0.77459667, -0.25819889],
       [ 0.        ,  0.57735027]])

In [96]:
np.linalg.cholesky(K_inv)

array([[ 1.0754151 ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.39562311,  1.0754151 ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.39562311,  1.0754151 ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.39562311,  1.0754151 ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.39562311,  1.        ]])

In [84]:
X = np.array([[2,1],[1,3]])
inv_X = np.linalg.inv(X)
chol_X = np.linalg.cholesky(X)
chol_inv_X = np.linalg.cholesky(inv_X)
inv_chol_X = np.linalg.inv(chol_X)
psuedo_chol_inv_X = inv_chol_X.T
print(X)
print(np.dot(chol_X,chol_X.T))
print(inv_X)
print(np.dot(chol_inv_X,chol_inv_X.T))
print(inv_chol_X)
print(chol_inv_X)
print(np.dot(psuedo_chol_inv_X,psuedo_chol_inv_X.T))
print(np.dot(chol_inv_X,chol_inv_X.T))

[[2 1]
 [1 3]]
[[2. 1.]
 [1. 3.]]
[[ 0.6 -0.2]
 [-0.2  0.4]]
[[ 0.6 -0.2]
 [-0.2  0.4]]
[[ 0.70710678  0.        ]
 [-0.31622777  0.63245553]]
[[ 0.77459667  0.        ]
 [-0.25819889  0.57735027]]
[[ 0.6 -0.2]
 [-0.2  0.4]]
[[ 0.6 -0.2]
 [-0.2  0.4]]


In [88]:
inv_chol_X

array([[ 0.70710678,  0.        ],
       [-0.31622777,  0.63245553]])

In [69]:
np.dot(chol_inv_X,chol_inv_X.T)

array([[ 0.6, -0.2],
       [-0.2,  0.4]])

In [70]:
np.dot(inv_chol_X.T,inv_chol_X)

array([[ 0.6, -0.2],
       [-0.2,  0.4]])

In [63]:
slinalg.inv(slinalg.cholesky(X).T)

array([[ 0.70710678, -0.        ],
       [-0.40824829,  0.81649658]])

In [64]:
slinalg.cholesky(slinalg.inv(X)).T

array([[ 0.81649658,  0.        ],
       [-0.40824829,  0.70710678]])

In [ ]:
def true_det(i):
    return np.linalg.det(K_inv[:i,:i])

u = np.exp(1.0/l)
i = 6
print(np.power(u*u/(u*u-1),i))
print(true_det(i))
print(c*true_det(i-1)-a*a*true_det(i-2))

In [ ]:
1-1.0/(u*u)

In [ ]:
K_inv[:1+1,:1+1]

In [ ]:
np.linalg.det(K_inv[:5,:5])

In [ ]:
-np.exp(1.0/l)/(np.exp(2.0/l)-1)

In [ ]:
np.exp(2.0/l)/(np.exp(2.0/l)-1)